In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# Dealing with Conflicts

This file shows how `Ply` deals with *shift/reduce* and *reduce/reduce* conflicts.

## Specification of the Scanner

We implement a minimal scanner for arithmetic expressions.

In [ ]:
import ply.lex as lex

tokens = [ 'NUMBER' ]

def t_NUMBER(t):
    r'0|[1-9][0-9]*'
    t.value = float(t.value)
    return t

literals = ['+', '-', '*', '/', '(', ')']

t_ignore  = ' \t'

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)

__file__ = 'main'

lexer = lex.lex()

## Specification of the Parser

In [ ]:
import ply.yacc as yacc

The *start variable* of our grammar is `expr`, but we don't have to specify that.  The default
start variable is the first variable that is defined.

In [ ]:
start = 'expr'

We can specify multiple expressions in a single rule.  In this case, we have used the `pass`statement 
as we just want to generate some conflicts. 

In [ ]:
def p_expr(p):
    """
    expr : expr '+' expr
         | expr '-' expr
         | expr '*' expr
         | expr '/' expr
         | '(' expr ')'
         | NUMBER            
    """
    pass

In [ ]:
def p_error(p):
    if p:
        print(f'Syntax error at {p.value}.')
    else:
        print('Syntax error at end of input.')

Setting the optional argument `write_tables` to `False` <B style="color:red">is required</B> to prevent an obscure bug where the parser generator tries  to read an empty parse table.

In [ ]:
parser = yacc.yacc(write_tables=False, debug=True)

Let's look at the action table that is generated.  Conflicts are always resolved in favour of shifting.

In [ ]:
!type parser.out

In [ ]:
!cat parser.out